# Import Lib

In [1]:
import pandas as pd
import os
import psycopg2
import numpy as np
import dask.dataframe as dd
import random
# import sqlalchemy as sql

# Create Connection to PostgreSQL

In [2]:
# Create Connection to postgresql

conn = psycopg2.connect(
    host="localhost",
    database="musicbrainz_db",
    user="musicbrainz",
    port=5432)

cursor = conn.cursor()

# psql -h localhost -p 25432 -U musicbrainz musicbrainz_db

# engine = sql.create_engine('postgresql:///musicbrainz_db')
# connection = engine.connect()
print("connection successful")

connection successful


In [14]:
#Load up MLHD Data Folder

filelist = os.listdir('/data/mlhd/00')

path = lambda name: '/data/mlhd/00/{}'.format(name)
file_paths = [path(name) for name in filelist]

first_10 = file_paths[:10]

In [12]:
# #random folder selector
# master_folder = os.listdir('/data/mlhd')
# folder_list = random.sample(master_folder, 10)

# #random file selection
# random_file_paths = []
# for folder in folder_list:
#     random_file = random.sample(os.listdir('/data/mlhd/'+(folder)), 1)[0]
# #     print(random_file)
#     path = '/data/mlhd/'+folder+'/'+random_file
#     random_file_paths.append(path)

# random_file_paths 

['/data/mlhd/4d/4d93959d-f872-42a4-b262-69f6800d5e42.txt.gz',
 '/data/mlhd/76/760d28ab-0950-428d-bdb4-de3a124336f9.txt.gz',
 '/data/mlhd/b4/b4666dd6-9ae6-4ef7-b40f-a1f4c154242f.txt.gz',
 '/data/mlhd/d0/d009df63-b2f9-4864-8664-5decd4dad443.txt.gz',
 '/data/mlhd/55/55bf8013-ebcf-4241-996f-15b0af947638.txt.gz',
 '/data/mlhd/d7/d75c3092-fcd6-4b02-98f6-2ab6acb27471.txt.gz',
 '/data/mlhd/d3/d39ee1d8-b6c2-42e0-a7d4-5dd1fd4fbd54.txt.gz',
 '/data/mlhd/f6/f6beb986-45a3-4e3d-a976-9b150b555f3c.txt.gz',
 '/data/mlhd/35/3531c6c1-7784-46ff-b5c1-4d72d395ed27.txt.gz',
 '/data/mlhd/e7/e79c358c-6223-4c1a-b84b-41ceaa592574.txt.gz']

In [17]:
# fetch df

# Use dask.dataframe since it's ~ 14x faster than pandas dataframe! 

def fetch_df_dd(pth):
    df = dd.read_csv(pth, sep='\t', names=['timestamp', 'artist-MBID', 'release-MBID', 'recording-MBID'], blocksize=None)
    df = df[-df['recording-MBID'].isna()]
    return df

df = fetch_df_dd(file_paths[0])
print(len(df))
df.head()

22963


,timestamp,artist-MBID,release-MBID,recording-MBID
0,1117464483,149e6720-4e4a-41a4-afca-6d29083fc091,549c0ea1-f42f-41f1-8276-e9bb8404fd3d,39072afe-b552-41bb-9660-8e431d53dfb0
1,1117505284,690c818f-a578-4e04-834a-a94dac6e3759,6d955eac-73fa-3cfe-bb9d-e5c2d0f5599a,2f6735c2-332b-4b05-85ec-b22eb27160b7
2,1117505646,14cd43ce-306b-463d-ad84-984c8f7b73ed,765c3615-9f26-4c9c-bcfc-1ff2a48d867c,0bd2efc9-96b3-4adf-b758-fd41b325d0b6
3,1117505758,14cd43ce-306b-463d-ad84-984c8f7b73ed,765c3615-9f26-4c9c-bcfc-1ff2a48d867c,98b7d4a6-7586-44ab-82e7-9b4a3095fe4b
4,1117506018,1a04561c-10dc-4a2e-b94c-19718ed1e0c7,ca42c729-e0ac-45bc-80ef-eff92b3da9c7,013f2240-5304-40d0-8b91-f1c365b59c35


In [9]:
# A function to take single mbid as input and: 
# 1. cross check with recording table. If present, return the SAME recording-mbid from recording table.
# 2. If not in recording table, check in track table. If present, return 
# 3. If not in recording/track table, check if it exists in the recording_gid_redirect table
# 4. If not in any of the above tables, add the mbid to bad_entries list.

# Check if a recording-MBID exists in the "recording" table. If exists, return the mbid. If not, return None
def check_in_recording(rec_mbid):
    cursor.execute("SELECT gid FROM recording WHERE gid=%(id)s;", {'id':rec_mbid})
    fetch = cursor.fetchone()
    
    if type(fetch) == tuple:
        return fetch[0]
    else:
        count_not_recording.add(rec_mbid)
        return None

# If MBID in recording_gid_redirect, return the redirected MBID. Else, return None
def check_in_redirect(rec_mbid):
    cursor.execute("select r.gid from recording r join recording_gid_redirect rgr on rgr.new_id = r.id where rgr.gid = %(id)s;", {'id':rec_mbid})
    fetch = cursor.fetchone()
    
    if type(fetch) == tuple:
        return fetch[0]
    else:
        count_not_redirect.add(rec_mbid)
        return None


# Check if a recording-MBID exists in the "track" table. If not, return None. 
# If yes, append the track-mbid to the list track_mbids

track_mbids = []
def check_in_track(rec_mbid):
    cursor.execute("SELECT gid FROM track WHERE gid=%(id)s;", {'id':rec_mbid})
    fetch = cursor.fetchone()
    
    if type(fetch) == tuple:
        track_mbids.add(rec_mbid)
        return fetch[0]
    else:
        count_not_track.add(rec_mbid)
        return None
    
# Function to validate MBIDs and replace MBIDS through a chain of functions.
def check_validity(rec_mbid):
    x = check_in_recording(rec_mbid)
    
    if x != None:
        return x
    else:
        x = check_in_redirect(rec_mbid)
    
        if x != None:
            return x
        else:            
            x = check_in_track(rec_mbid)
            
            if x!= None:
                return x
            else:
                return None
            
# Simple function to take single mbid as input and:
# Find its canonical MBID
def get_canon(rec_mbid):
    cursor.execute("SELECT canonical_recording_mbid FROM mapping.canonical_recording WHERE recording_mbid=%(id)s;", {'id':rec_mbid})
    if cursor.fetchone()!=None:
        count_not_canonical.add(rec_mbid)
        return cursor.fetchone()
    else:
        return rec_mbid

In [10]:
# Declaring count sets
count_not_recording, count_not_redirect, count_not_track, count_misc, count_not_canonical = set(), set(), set(), set(), set()

# Driver function to check validity and get canononical MBID.
def driver_function(rec_mbid):
    x = check_validity(rec_mbid)
    if x != None:
        return get_canon(rec_mbid)
    else:
        return None

In [11]:
df2 = df['recording-MBID'].map(driver_function, meta=('x', str)).compute()
# track_mbids

In [12]:
print("total rows in dataset =", len(df))
print("number of Null values in appended to the dataset =", df2.isna().sum())
print("number of values not present in the table 'recording' =",len(count_not_recording))
print("number of values not present niether in the table 'recording' nor in 'redirect' =",len(count_not_redirect))
print("num of values niether in recording, nor redirect, nor track =",len(count_not_track))
print("number of track-MBIDs in the recording-MBID column =",len(track_mbids))
print("num of broken MBIDs =", len(count_not_redirect.intersection(count_not_track)))
print("num of non-canonical IDs found in recording-mbid table =", len(count_not_canonical))

total rows in dataset = 9700
number of Null values in appended to the dataset = 13
number of values not present in the table 'recording' = 513
number of values not present niether in the table 'recording' nor in 'redirect' = 7
num of values niether in recording, nor redirect, nor track = 7
number of track-MBIDs in the recording-MBID column = 0
num of broken MBIDs = 7
num of non-canonical IDs found in recording-mbid table = 0


### Copied result from the above computation ^

total rows in dataset = 9700

number of Null values in appended to the dataset = 13

number of values not present in the table 'recording' = 513

number of values not present niether in the table 'recording' nor in 'redirect' = 7

num of values niether in recording, nor redirect, nor track = 7

number of track-MBIDs in the recording-MBID column = 0

num of broken MBIDs = 7

num of non-canonical IDs found in recording-mbid table = 0